In [1]:
import collections
import itertools  
import matplotlib.pyplot as plt
import networkx as nx
import os
import tellurium as te
import tesbml
import os, sys
cwd = os.getcwd()
print(cwd)

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [2]:
parent_dir = os.path.abspath(os.path.join(cwd, os.pardir))
print(parent_dir)
os.chdir(parent_dir)
print(os.getcwd())

/Users/woosubshin/Desktop/ModelEngineering/SBMLLint
/Users/woosubshin/Desktop/ModelEngineering/SBMLLint


In [3]:
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule
from SBMLLint.common.reaction import Reaction
from SBMLLint.common.simple_sbml import SimpleSBML
from SBMLLint.games import print_model as pm
from SBMLLint.games.som import SOM

In [4]:
num = 17
format_num = format(num, '03d')
document = tesbml.readSBML(os.path.join(os.getcwd(), 'SBMLLint/games/data/curated_' + format_num + '.xml'))
model = document.getModel()
pm.print_model(model)

<Model Hoefnagel2002_PyruvateBranches "Hoefnagel2002_PyruvateBranches">
R1: ADP + NAD + halfglucose -> ATP + NADH + pyruvate;
R2: NADH + pyruvate -> NAD + lactate;
R3: NAD + pyruvate + CoA -> NADH + AcCoA;
R4: AcCoA + PO4 -> CoA + AcP;
R5: ADP + AcP -> ATP + Ac;
R6: NADH + AcCoA -> NAD + CoA + AcO;
R7: NADH + AcO -> NAD + EtOH;
R8: pyruvate -> AcLac;
R9: AcLac -> AcetoinIn;
R10: AcetoinIn -> AcetoinOut;
R11: NADH + AcetoinIn -> NAD + Butanediol;
R12: ATP -> ADP;
R13: NADH + O2 -> NAD;
R14: AcLac -> AcetoinIn;


In [5]:
simple = SimpleSBML(model)

In [6]:
print(simple._getReactions())
len(simple._getReactions())

[<Reaction R1>, <Reaction R2>, <Reaction R3>, <Reaction R4>, <Reaction R5>, <Reaction R6>, <Reaction R7>, <Reaction R8>, <Reaction R9>, <Reaction R10>, <Reaction R11>, <Reaction R12>, <Reaction R13>, <Reaction R14>]


14

In [7]:
Reaction.reactions = []
Reaction.initialize(simple)
print(Reaction.reactions)
len(Reaction.reactions)
# makes sense because R9 and R14 are same;

[ADP + NAD + halfglucose -> ATP + NADH + pyruvate, NADH + pyruvate -> NAD + lactate, CoA + NAD + pyruvate -> AcCoA + NADH, AcCoA + PO4 -> AcP + CoA, ADP + AcP -> ATP + Ac, AcCoA + NADH -> AcO + CoA + NAD, AcO + NADH -> EtOH + NAD, pyruvate -> AcLac, AcLac -> AcetoinIn, AcetoinIn -> AcetoinOut, AcetoinIn + NADH -> Butanediol + NAD, ATP -> ADP, NADH + O2 -> NAD]


13

In [8]:
Reaction.reactions

[ADP + NAD + halfglucose -> ATP + NADH + pyruvate,
 NADH + pyruvate -> NAD + lactate,
 CoA + NAD + pyruvate -> AcCoA + NADH,
 AcCoA + PO4 -> AcP + CoA,
 ADP + AcP -> ATP + Ac,
 AcCoA + NADH -> AcO + CoA + NAD,
 AcO + NADH -> EtOH + NAD,
 pyruvate -> AcLac,
 AcLac -> AcetoinIn,
 AcetoinIn -> AcetoinOut,
 AcetoinIn + NADH -> Butanediol + NAD,
 ATP -> ADP,
 NADH + O2 -> NAD]

In [9]:
# sbml_reaction1 = model.getReaction(0)
# sbml_reaction2 = model.getReaction(11)
# Reaction.reactions = []
# Reaction(sbml_reaction1)
# Reaction(sbml_reaction2)
# Reaction.reactions

In [10]:
print(Molecule.molecules)

[ADP, NAD, halfglucose, ATP, NADH, pyruvate, lactate, CoA, AcCoA, PO4, AcP, Ac, AcO, EtOH, AcLac, AcetoinIn, AcetoinOut, Butanediol, O2]


In [456]:
SOM.initialize(Molecule.molecules)
print(SOM.soms)

[{ADP}, {NAD}, {halfglucose}, {ATP}, {NADH}, {pyruvate}, {lactate}, {CoA}, {AcCoA}, {PO4}, {AcP}, {Ac}, {AcO}, {EtOH}, {AcLac}, {AcetoinIn}, {AcetoinOut}, {Butanediol}, {O2}]


In [434]:
adp = SOM.findSOM(Molecule.getMolecule('ADP'))
atp = SOM.findSOM(Molecule.getMolecule('ATP'))
print(adp)
print(atp)

{ADP}
{ATP}


In [333]:
a = nx.DiGraph()
a.add_nodes_from(SOM.soms)
a.nodes()
a.add_edge(adp, atp)
print(a.nodes)
print(a.edges)

[{ADP}, {NAD}, {halfglucose}, {ATP}, {NADH}, {pyruvate}, {lactate}, {CoA}, {AcCoA}, {PO4}, {AcP}, {Ac}, {AcO}, {EtOH}, {AcLac}, {AcetoinIn}, {AcetoinOut}, {Butanediol}, {O2}]
[({ADP}, {ATP})]


In [435]:
class Arc(object):
    
    def __init__(self, source, destination, reaction):
        """
        :param SOM source:
        :param SOM destination:
        :param Reaction reaction:
        """
        self.source = source
        self.destination = destination
        self.reaction = reaction

In [478]:
SOM.initialize(Molecule.molecules)
print(SOM.soms)

[{ADP}, {NAD}, {halfglucose}, {ATP}, {NADH}, {pyruvate}, {lactate}, {CoA}, {AcCoA}, {PO4}, {AcP}, {Ac}, {AcO}, {EtOH}, {AcLac}, {AcetoinIn}, {AcetoinOut}, {Butanediol}, {O2}]


In [479]:
# maybe we can have multiple MESGraphs.. for each subgraph is a connected subgraph
class MESGraph(nx.DiGraph):
    # possibly can have all = [] here. 
    all = []

    def __init__(self, som=None):
        
        super(MESGraph, self).__init__()
        self.add_node(som)
        self.arcs = set()
        self.identifier = self.makeId()
        self.__class__.all.append(self)

    def __repr__(self):
        return self.identifier
    
    def makeId(self):
        identifier = ";"
        for som in list(self.nodes):
            identifier = identifier + som.identifier + "->"
        return identifier
    
    @classmethod
    def processUniUniReaction(cls, reaction):
        """
        :param list-Reaction reactions:
        """
        if reaction.category != cn.REACTION_1_1:
            pass
        else:
            reactant_som = SOM.findSOM(reaction.reactants[0].molecule)
            product_som = SOM.findSOM(reaction.products[0].molecule)
            reactant_graph = cls.findGraph(reaction.reactants[0].molecule)
            product_graph = cls.findGraph(reaction.products[0].molecule)
            
            new_som = SOM.merge(reaction)
            #print("new som is: ", new_som)
            
            ## need to do a little more to merge if the graph is not a one-som graph
            # reactant_graph.remove_node(reactant_som)
            # product_graph.remove_node(product_som)
            cls.removeGraph(reactant_som)
            cls.removeGraph(product_som)
            
            # below, or need mergeGraph
            MESGraph(new_som)
            return new_som
            
    @classmethod
    def removeGraph(cls, som):
        """
        :param SOM som:
        """
        for graph in cls.all:
            if som in graph.nodes:
                cls.all.remove(graph)
                return True
        return False
        
    
    @classmethod
    def findGraph(cls, molecule):
        """
        :param Molecule molecule:
        :return MESGraph/False:
        """
        som = SOM.findSOM(molecule)
        for graph in cls.all:
            for node in graph.nodes:
                if node == som:
                    return graph
        return False
        
    @classmethod        
    def analyze(cls, reactions):
        """
        :param list-Reaction reactions:
        """
        uniuni = []
        unimulti = []
        multiuni = []
        multimulti = []
        for reaction in reactions:
            if reaction.category == cn.REACTION_1_1:
                uniuni.append(reaction)
            elif reaction.category == cn.REACTION_1_n:
                unimulti.append(reaction)
            elif reaction.category == cn.REACTION_n_1:
                multiuni.append(reaction)
            else:
                multimulti.append(reaction)
            
#         self.uniuni = uniuni
#         self.unimulti = unimulti
#         self.multiuni = multiuni
#         self.multimulti = multimulti
        for reaction in uniuni:
            cls.processUniUniReaction(reaction)
            print("We used + ", reaction)
        #return self.soms
        
    @classmethod
    def initialize(cls, soms):
        """
        :param list-SOM soms:
        """
        cls.all = []
        for som in soms:
            MESGraph(som)
            

In [480]:
MESGraph.initialize(SOM.soms)
print(MESGraph.all)

[;{ADP}->, ;{NAD}->, ;{halfglucose}->, ;{ATP}->, ;{NADH}->, ;{pyruvate}->, ;{lactate}->, ;{CoA}->, ;{AcCoA}->, ;{PO4}->, ;{AcP}->, ;{Ac}->, ;{AcO}->, ;{EtOH}->, ;{AcLac}->, ;{AcetoinIn}->, ;{AcetoinOut}->, ;{Butanediol}->, ;{O2}->]


In [481]:
MESGraph.all

[;{ADP}->,
 ;{NAD}->,
 ;{halfglucose}->,
 ;{ATP}->,
 ;{NADH}->,
 ;{pyruvate}->,
 ;{lactate}->,
 ;{CoA}->,
 ;{AcCoA}->,
 ;{PO4}->,
 ;{AcP}->,
 ;{Ac}->,
 ;{AcO}->,
 ;{EtOH}->,
 ;{AcLac}->,
 ;{AcetoinIn}->,
 ;{AcetoinOut}->,
 ;{Butanediol}->,
 ;{O2}->]

In [482]:
MESGraph.analyze(Reaction.reactions)
MESGraph.all

new som is:  {AcLac, pyruvate}
We used +  pyruvate -> AcLac
new som is:  {AcLac, AcetoinIn, pyruvate}
We used +  AcLac -> AcetoinIn
new som is:  {AcLac, AcetoinIn, AcetoinOut, pyruvate}
We used +  AcetoinIn -> AcetoinOut
new som is:  {ADP, ATP}
We used +  ATP -> ADP


[;{NAD}->,
 ;{halfglucose}->,
 ;{NADH}->,
 ;{lactate}->,
 ;{CoA}->,
 ;{AcCoA}->,
 ;{PO4}->,
 ;{AcP}->,
 ;{Ac}->,
 ;{AcO}->,
 ;{EtOH}->,
 ;{Butanediol}->,
 ;{O2}->,
 ;{AcLac, AcetoinIn, AcetoinOut, pyruvate}->,
 ;{ADP, ATP}->]

In [476]:
reaction1 = Reaction.reactions[7]
reaction1
print(reaction1.reactants[0].molecule)
print(reaction1.products[0].molecule)

pyruvate
AcLac


In [468]:
MESGraph.processUniUniReaction(reaction1)

Found it!:
Found it!:
new som is:  {AcLac, pyruvate}


{AcLac, pyruvate}

In [469]:
MESGraph.all

[;{ADP}->,
 ;{NAD}->,
 ;{halfglucose}->,
 ;{ATP}->,
 ;{NADH}->,
 ;{lactate}->,
 ;{CoA}->,
 ;{AcCoA}->,
 ;{PO4}->,
 ;{AcP}->,
 ;{Ac}->,
 ;{AcO}->,
 ;{EtOH}->,
 ;{AcetoinIn}->,
 ;{AcetoinOut}->,
 ;{Butanediol}->,
 ;{O2}->,
 ;{AcLac, pyruvate}->]

In [350]:
reaction1

pyruvate -> AcLac

In [382]:
m

{ADP}->

In [392]:
m.nodes

NodeView(({ADP},))

In [387]:
list(m.nodes)[0] = list(n.nodes)[0]

In [390]:
m.nodes

NodeView(({ADP},))

In [393]:
k = m

In [394]:
k

{ADP}->

In [416]:
print(MESGraph.all)
adp = MESGraph.all[0]
nad = MESGraph.all[1]
k.remove_node(nad)

[{ADP}->, {NAD}->, {halfglucose}->, {ATP}->, {NADH}->, {pyruvate}->, {lactate}->, {CoA}->, {AcCoA}->, {PO4}->, {AcP}->, {Ac}->, {AcO}->, {EtOH}->, {AcLac}->, {AcetoinIn}->, {AcetoinOut}->, {Butanediol}->, {O2}->]


NetworkXError: The node  is not in the digraph.

In [417]:
k.add_node(nad)

In [418]:
k.identifier = k.makeId()

In [420]:
k

{NAD}->->

In [425]:
pyruvate = Molecule.molecules[5]
set({pyruvate})

{pyruvate}

In [426]:
m

{NAD}->->